In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# EDA

In [2]:
df = pd.read_csv("get_around_pricing_project.csv")
df.head()

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [3]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [4]:
df.head()

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [5]:
df.shape

(4843, 14)

In [6]:
df.describe(include='all')

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4843,4.843000e+03,4843.00000,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843.000000
unique,28,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,969,NaN,NaN,4641,1633,1606,2662,3839,3865,3881,2613,3674,4514,NaN
mean,NaN,1.409628e+05,128.98823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.214536
std,NaN,6.019674e+04,38.99336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.568268
min,NaN,-6.400000e+01,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000
25%,NaN,1.029135e+05,100.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.000000
50%,NaN,1.410800e+05,120.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.000000
75%,NaN,1.751955e+05,135.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.000000


In [7]:
# on remarque des valeurs abbérantes dans la colonne "mileage" et "engine_power" sur le minimum

df[(df['mileage'] <= 0) | (df['engine_power'] <= 0)]

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
2938,Renault,-64,230,diesel,black,sedan,True,True,False,True,False,False,True,274
3765,Nissan,81770,0,diesel,white,suv,False,False,False,False,False,False,False,108


In [8]:
# on va supprimer ces lignes ce sont clairement des valeurs abbérantes

df = df.drop(df[(df['mileage'] <= 0)|(df['engine_power'] <= 0)].index)

In [9]:
df.describe(include='all')

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4841,4.841000e+03,4841.000000,4841,4841,4841,4841,4841,4841,4841,4841,4841,4841,4841.000000
unique,28,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,969,NaN,NaN,4639,1632,1606,2661,3838,3863,3880,2611,3672,4513,NaN
mean,NaN,1.410042e+05,128.994010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.185705
std,NaN,6.016901e+04,38.930253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.502751
min,NaN,4.760000e+02,25.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000
25%,NaN,1.030340e+05,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.000000
50%,NaN,1.410890e+05,120.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.000000
75%,NaN,1.752170e+05,135.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.000000


In [10]:
#missing values
df.isnull().sum()

model_key                    0
mileage                      0
engine_power                 0
fuel                         0
paint_color                  0
car_type                     0
private_parking_available    0
has_gps                      0
has_air_conditioning         0
automatic_car                0
has_getaround_connect        0
has_speed_regulator          0
winter_tires                 0
rental_price_per_day         0
dtype: int64

In [11]:
import plotly.express as px

px.histogram(df, x="rental_price_per_day", nbins=50)

In [12]:
df[df['rental_price_per_day'] > 400]

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
4753,BMW,72515,135,diesel,blue,suv,False,False,True,False,False,False,False,422


In [13]:
px.histogram(df, x='model_key').update_xaxes(categoryorder='total descending')

On remarque une multitude de marque de véhicule qui sont insignifiantes sur la location, nous allons les regrouper dans une même catégorie "Others" \
Cela nous permettra de rééquilibrer le dataset avant l'entrainement du modèle de ML (linear Regression)

In [14]:
brand_count = df['model_key'].value_counts()
# on fixe un threshold minimum de 30 occurences
min_threshold = 30

# on garde que les marques qui ont plus de 30 occurences
rare_brands = brand_count[brand_count < min_threshold].index.to_list()

# on applique une fonction qui va remplacer dans la colonne "model_key" les marques qui ont moins de 30 occurences par "Others"
df['model_key'] = df['model_key'].apply(lambda x: 'Others' if x in rare_brands else x)
df['model_key'].value_counts()


model_key
Citroën       969
Renault       915
BMW           827
Peugeot       642
Audi          526
Nissan        274
Mitsubishi    231
Mercedes       97
Volkswagen     65
Others         53
Toyota         53
SEAT           46
Subaru         44
PGO            33
Opel           33
Ferrari        33
Name: count, dtype: int64

In [15]:
px.histogram(df, x='model_key').update_xaxes(categoryorder='total descending')

In [16]:
px.histogram(df, x='car_type').update_xaxes(categoryorder='total descending')

In [17]:
df['car_type'].value_counts()

car_type
estate         1606
sedan          1167
suv            1057
hatchback       699
subcompact      117
coupe           104
convertible      47
van              44
Name: count, dtype: int64

In [18]:
px.histogram(df, x='fuel')

# MACHINE LEARNING

In [25]:
target_variable = "rental_price_per_day"
X = df.drop(target_variable, axis=1)
y = df[target_variable]
print(y)


0       106
1       264
2       101
3       158
4       183
       ... 
4838    121
4839    132
4840    130
4841    151
4842    124
Name: rental_price_per_day, Length: 4841, dtype: int64


In [26]:
# séparation des données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#preprocessing

numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(exclude=[np.number]).columns

In [27]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
    ])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [28]:
regressor = LinearRegression()

regressor.fit(X_train, y_train)
y_train_pred = regressor.predict(X_train)
print(y_train_pred)

y_test_pred = regressor.predict(X_test)
print(y_test_pred)

[ 97.00031148 118.52383736 105.71188515 ... 125.45978902 144.77912201
 138.66090138]
[116.12168583 145.39157911 170.72397533  86.70951643 102.57329143
 104.64642632  99.53271708 133.96238544 114.958665   107.32958272
  87.79676942 140.41031305 101.90577282 111.40066031 128.75381658
 107.625601   128.55956741  78.24817255  91.10866646 101.10403868
  92.83809235 115.74381936 100.79308561 112.6762977   99.52007014
 135.94114477 103.3197853  100.06302619 167.29872025  98.35348082
  86.67512195 132.22102628 108.60680271 146.82253182  99.50650188
 122.37563405 145.51554823 227.29779424 123.66189581 138.20817537
 114.67068566 175.64793052 132.65822492 121.72881072 147.10961845
 101.14290924 101.1597521   83.01724171 114.63710817 114.22962991
  94.83641333 123.52290609  99.45169128 127.60364171 123.1303139
  89.72260602 109.42853937 134.65425978 149.69984054 128.07524464
 134.54645426 118.0761125  142.40373554  98.71779857  92.57061987
 129.71734775  98.64357086 152.94617532 103.62455268 137.6

In [29]:
#evaluation
regressor.score(X_train, y_train), regressor.score(X_test, y_test)

(0.7058967190013714, 0.7054833788323966)

In [30]:
r2_score_train = r2_score(y_train, y_train_pred)
r2_score_test = r2_score(y_test, y_test_pred)
print(f"R2 score train: {r2_score_train}")
print(f"R2 score test: {r2_score_test}")

R2 score train: 0.7058967190013714
R2 score test: 0.7054833788323966
